In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 45.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a437c3f37e24d7105acebd0ad45b21e82d01cf716e692990305a05f2dbad3ca2
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pyspark.sql.types as typ
from pyspark.ml import Pipeline
import pyspark.ml.classification as cl
import pyspark.ml.evaluation as ev
import pandas as pd
import numpy as np
import os

In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder \
    .appName("GenericAppName") \
        .getOrCreate()

sc = spark.sparkContext

# If you need to use SQL context, you need to initialize it here.
sqlContext = SQLContext(sc)


sc

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


<SparkContext master=local[*] appName=GenericAppName>

# Read the preprocessed data

In [ ]:
co_p = pd.read_csv('co_pitts.txt',header=None)
o3_p = pd.read_csv('o3_pitts.txt',header=None)
pm10_p = pd.read_csv('pm10_pitts.txt',header=None)
pm25_p = pd.read_csv('pm25_pitts.txt',header=None)
so2_p = pd.read_csv('so2_pitts.txt',header=None)
aqi_o = pd.read_csv('aqi_pitts.txt',header=None)

co_1 = co_p.iloc[1:,:].reset_index(drop=True)
co_2 = co_p.iloc[2:,:].reset_index(drop=True)
co_3 = co_p.iloc[3:,:].reset_index(drop=True)
o3_1 = o3_p.iloc[1:,:].reset_index(drop=True)
o3_2 = o3_p.iloc[2:,:].reset_index(drop=True)
o3_3 = o3_p.iloc[3:,:].reset_index(drop=True)

pm10_1 = pm10_p.iloc[1:,:].reset_index(drop=True)
pm10_2 = pm10_p.iloc[2:,:].reset_index(drop=True)
pm10_3 = pm10_p.iloc[3:,:].reset_index(drop=True)
pm25_1 = pm25_p.iloc[1:,:].reset_index(drop=True)
pm25_2 = pm25_p.iloc[2:,:].reset_index(drop=True)
pm25_3 = pm25_p.iloc[3:,:].reset_index(drop=True)

so2_1 = so2_p.iloc[1:,:].reset_index(drop=True)
so2_2 = so2_p.iloc[2:,:].reset_index(drop=True)
so2_3 = so2_p.iloc[3:,:].reset_index(drop=True)
aqi_1 = aqi_o.iloc[1:,:].reset_index(drop=True)
aqi_2 = aqi_o.iloc[2:,:].reset_index(drop=True)
aqi_3 = aqi_o.iloc[3:,:].reset_index(drop=True)

In [ ]:
co_p = co_p.iloc[:-3,:].reset_index(drop=True)
o3_p = o3_p.iloc[:-3,:].reset_index(drop=True)
pm10_p = pm10_p.iloc[:-3,:].reset_index(drop=True)
pm25_p = pm25_p.iloc[:-3,:].reset_index(drop=True)
so2_p = so2_p.iloc[:-3,:].reset_index(drop=True)
aqi = aqi_o.iloc[:-3,:].reset_index(drop=True)

co_1 = co_1.iloc[:-2,:].reset_index(drop=True)
o3_1 = o3_1.iloc[:-2,:].reset_index(drop=True)
pm10_1 = pm10_1.iloc[:-2,:].reset_index(drop=True)
pm25_1 = pm25_1.iloc[:-2,:].reset_index(drop=True)
so2_1 = so2_1.iloc[:-2,:].reset_index(drop=True)
aqi_1 = aqi_1.iloc[:-2,:].reset_index(drop=True)

co_2 = co_2.iloc[:-1,:].reset_index(drop=True)
o3_2 = o3_2.iloc[:-1,:].reset_index(drop=True)
pm10_2 = pm10_2.iloc[:-1,:].reset_index(drop=True)
pm25_2 = pm25_2.iloc[:-1,:].reset_index(drop=True)
so2_2 = so2_2.iloc[:-1,:].reset_index(drop=True)
aqi_2 = aqi_2.iloc[:-1,:].reset_index(drop=True)

In [ ]:
from functools import reduce
dfs = [co_3,o3_3,pm10_3,pm25_3,so2_3,aqi_3,co_2,o3_2,pm10_2,pm25_2,so2_2,aqi_2,co_1,o3_1,pm10_1,pm25_1,so2_1,aqi_1,co_p,o3_p,pm10_p,pm25_p,so2_p,aqi]
df = reduce(lambda left, right: pd.merge(left, right, how='outer', left_index=True, right_index=True), dfs)

<ipython-input-7-13380ffd0b00>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = reduce(lambda left, right: pd.merge(left, right, how='outer', left_index=True, right_index=True), dfs)


In [ ]:
feature_cols = ['co_3','o3_3','pm10_3','pm25_3','so2_3','aqi_3','co_2','o3_2','pm10_2','pm25_2','so2_2','aqi_2','co_1','o3_1','pm10_1','pm25_1','so2_1','aqi_1','co','o3','pm10','pm25','so2','aqi']
train_cols = ['co_3','o3_3','pm10_3','pm25_3','so2_3','aqi_3','co_2','o3_2','pm10_2','pm25_2','so2_2','aqi_2','co_1','o3_1','pm10_1','pm25_1','so2_1','aqi_1']
drop_cols = ['co_3','o3_3','pm10_3','pm25_3','so2_3','aqi_3','co_2','o3_2','pm10_2','pm25_2','so2_2','aqi_2','co_1','o3_1','pm10_1','pm25_1','so2_1','aqi_1','co','o3','pm10','pm25','so2']

In [ ]:
df.columns = feature_cols

In [ ]:
df

,co_3,o3_3,pm10_3,pm25_3,so2_3,aqi_3,co_2,o3_2,pm10_2,pm25_2,...,pm10_1,pm25_1,so2_1,aqi_1,co,o3,pm10,pm25,so2,aqi
0,0.741667,0.024,5.000000,10.577476,0.282609,22.0,0.820833,0.010,13.000000,10.577476,...,16.000000,10.577476,1.356522,19.0,0.600000,0.027,9.000000,10.577476,1.556522,25.0
1,0.700000,0.028,12.000000,10.577476,0.260870,26.0,0.741667,0.024,5.000000,10.577476,...,13.000000,10.577476,0.543478,9.0,0.679167,0.021,16.000000,10.577476,1.356522,19.0
2,0.700000,0.028,8.000000,10.577476,0.930435,26.0,0.700000,0.028,12.000000,10.577476,...,5.000000,10.577476,0.282609,22.0,0.820833,0.010,13.000000,10.577476,0.543478,9.0
3,0.729167,0.029,16.000000,10.577476,0.978261,27.0,0.700000,0.028,8.000000,10.577476,...,12.000000,10.577476,0.260870,26.0,0.741667,0.024,5.000000,10.577476,0.282609,22.0
4,0.700000,0.025,15.000000,10.577476,1.530435,23.0,0.729167,0.029,16.000000,10.577476,...,8.000000,10.577476,0.930435,26.0,0.700000,0.028,12.000000,10.577476,0.260870,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,0.220833,0.017,15.074687,12.300000,0.223810,16.0,0.212500,0.026,15.074687,14.200000,...,15.074687,5.600000,0.252174,24.0,0.329167,0.034,15.074687,6.000000,0.730435,31.0
2550,0.241667,0.025,15.074687,9.600000,0.227273,23.0,0.220833,0.017,15.074687,12.300000,...,15.074687,14.200000,0.252174,24.0,0.258333,0.026,15.074687,5.600000,0.252174,24.0
2551,0.275000,0.020,15.074687,9.700000,0.178261,19.0,0.241667,0.025,15.074687,9.600000,...,15.074687,12.300000,0.223810,16.0,0.212500,0.026,15.074687,14.200000,0.252174,24.0
2552,0.300000,0.010,15.074687,14.700000,0.265217,9.0,0.275000,0.020,15.074687,9.700000,...,15.074687,9.600000,0.227273,23.0,0.220833,0.017,15.074687,12.300000,0.223810,16.0


# Create a pipeline for data process, get df_train, df_test

In [ ]:
df_spark = spark.createDataFrame(df)

In [ ]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

from pyspark.ml.regression import LinearRegression

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()
    def _transform(self, dataset):
        output_df = dataset
        for col_name in output_df.columns:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))
        return output_df

class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

# cast the columns as appropriate types (DoubleType)
stage_typecaster = FeatureTypeCaster()

# Stage where all relevant features are assembled into a vector 
stage_vector_assembler = VectorAssembler(inputCols=train_cols, outputCol="vectorized_features")

# Stage where we scale the columns
stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

# Removing all unnecessary columbs, only keeping the 'features' and 'aqi' columns
stage_column_dropper = ColumnDropper(columns_to_drop = drop_cols + ['vectorized_features'])
      
pipeline = Pipeline(stages=[stage_typecaster,stage_vector_assembler,stage_scaler,stage_column_dropper])

In [ ]:
pipeline_model = pipeline.fit(df_spark)
df_transformed = pipeline_model.transform(df_spark)

df_train, df_test = df_transformed.randomSplit(weights=[0.8,0.2], seed=2022)

In [ ]:
df_train.show()

+----+--------------------+
| aqi|            features|
+----+--------------------+
| 0.0|[0.94786709227486...|
| 0.0|[1.32701392918481...|
| 2.0|[1.61137405686727...|
| 2.0|[1.64297215425535...|
| 3.0|[0.91626899488679...|
| 3.0|[2.36966773068717...|
| 5.0|[2.14849621484412...|
| 6.0|[1.20062912256925...|
| 7.0|[2.81200317943653...|
| 9.0|[1.57977595947920...|
| 9.0|[3.00157659789150...|
| 9.0|[5.30805571673927...|
|10.0|[1.83254557271032...|
|10.0|[2.30647911884776...|
|11.0|[2.87519937421268...|
|11.0|[4.89731078244124...|
|11.0|[8.81516395815629...|
|12.0|[2.21169240962027...|
|12.0|[2.27488102145968...|
|12.0|[3.07530549179701...|
+----+--------------------+
only showing top 20 rows



# Linear Regression with hyperparameter adjustment

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = RegressionEvaluator(labelCol='aqi', metricName='mse')

lr = LinearRegression(featuresCol='features',labelCol='aqi')

lr_param_grid = (ParamGridBuilder()
          .addGrid(lr.regParam, [0.001,0.01,0.1,1])
          .addGrid(lr.maxIter, [100,200,500,1000])
          .build())
                 
lr_cv = CrossValidator(estimator=lr, 
            estimatorParamMaps=lr_param_grid,
            evaluator=evaluator,
            parallelism=8,
            numFolds=5)
                 
lr_cv_model = lr_cv.fit(df_train)

In [ ]:
print("# after Cross-Validation")

print("Parameter: Mean Squard Error(MSE)")

lr_train_predictions = lr_cv_model.transform(df_train)
print('Train MSE of LinearRegression Model :', evaluator.evaluate(lr_train_predictions))

lr_test_predictions = lr_cv_model.transform(df_test)
print('Test MSE of LinearRegression Model :', evaluator.evaluate(lr_test_predictions))

# after Cross-Validation
Parameter: Mean Squard Error(MSE)
Train MSE of LinearRegression Model : 122.29050569537486
Test MSE of LinearRegression Model : 131.59471738853406


In [ ]:
lr_train_predictions.show()

+----+--------------------+------------------+
| aqi|            features|        prediction|
+----+--------------------+------------------+
| 0.0|[0.94786709227486...|22.356980939667327|
| 0.0|[1.32701392918481...|21.262690975376213|
| 2.0|[1.61137405686727...|  16.7439245669144|
| 2.0|[1.64297215425535...|27.938951667840943|
| 3.0|[0.91626899488679...|19.047069913591763|
| 3.0|[2.36966773068717...| 3.556679877709546|
| 5.0|[2.14849621484412...|16.977488702992314|
| 6.0|[1.20062912256925...| 22.43322741200717|
| 7.0|[2.81200317943653...| 25.76768801202417|
| 9.0|[1.57977595947920...|18.373856795464334|
| 9.0|[3.00157659789150...|23.294550060012174|
| 9.0|[5.30805571673927...| 20.57069764326437|
|10.0|[1.83254557271032...| 26.34224415481002|
|10.0|[2.30647911884776...|24.837539419611034|
|11.0|[2.87519937421268...| 30.88737517954587|
|11.0|[4.89731078244124...| 23.26488895569405|
|11.0|[8.81516395815629...|13.462329401841952|
|12.0|[2.21169240962027...|17.445828093577802|
|12.0|[2.2748

# Keras neural network


In [ ]:
import tensorflow as tf
from tensorflow import keras

to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

df_test_half, df_validate = df_test.randomSplit(weights=[0.5, 0.5], seed=2022)

df_train_pandas = df_train.withColumn('features', to_array('features')).toPandas()
df_validate_pandas = df_validate.withColumn('features', to_array('features')).toPandas()
df_test_half_pandas = df_test_half.withColumn('features', to_array('features')).toPandas()

x_train = tf.constant(np.array(df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(df_train_pandas['aqi'].values.tolist()))

x_validate = tf.constant(np.array(df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(df_validate_pandas['aqi'].values.tolist()))

x_test = tf.constant(np.array(df_test_half_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(df_test_half_pandas['aqi'].values.tolist()))

In [ ]:
#to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

#df_transformed_pandas = df_transformed.withColumn('features', to_array('features')).toPandas()

HP_WIDTH = [15,18,20,25]
HP_DEPTH = [3,5,7,10]

depth = []
width = []
loss = []
val_loss = []

for dep in HP_DEPTH:
    for wid in HP_WIDTH:
        model = keras.Sequential()
        for i in range(dep):
            model.add(keras.layers.Dense(wid,activation='relu'))
        model.add(keras.layers.Dense(1))   
        model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(0.01))

        history = model.fit(x_train, y_train, epochs = 100, verbose = 0,
                            validation_data = (x_validate, y_validate))
        loss.append(np.min(history.history["loss"]))
        val_loss.append(np.min(history.history["val_loss"]))
        depth.append(dep)
        width.append(wid)


        print('--- Depth is :',dep)
        print('--- Width is :',wid)
        print('--- loss(mean_squared_error,mse) for this trial is:',loss[-1])
        print('--- val_loss(mean_squared_error,mse) for this trial is:',val_loss[-1])
        print()

--- Depth is : 3
--- Width is : 15
--- loss(mean_squared_error,mse) for this trial is: 106.28331756591797
--- val_loss(mean_squared_error,mse) for this trial is: 110.77194213867188

--- Depth is : 3
--- Width is : 18
--- loss(mean_squared_error,mse) for this trial is: 109.43051147460938
--- val_loss(mean_squared_error,mse) for this trial is: 108.07666015625

--- Depth is : 3
--- Width is : 20
--- loss(mean_squared_error,mse) for this trial is: 108.24552917480469
--- val_loss(mean_squared_error,mse) for this trial is: 110.68148803710938

--- Depth is : 3
--- Width is : 25
--- loss(mean_squared_error,mse) for this trial is: 105.55078125
--- val_loss(mean_squared_error,mse) for this trial is: 111.8078842163086

--- Depth is : 5
--- Width is : 15
--- loss(mean_squared_error,mse) for this trial is: 109.71038055419922
--- val_loss(mean_squared_error,mse) for this trial is: 107.03189086914062

--- Depth is : 5
--- Width is : 18
--- loss(mean_squared_error,mse) for this trial is: 107.094322204

In [ ]:
best_wid = width[np.array(val_loss).argmin()]
best_dep = depth[np.array(val_loss).argmin()]

best_model = keras.Sequential()
for i in range(best_dep):
    best_model.add(keras.layers.Dense(best_wid,activation='relu'))
best_model.add(keras.layers.Dense(1))

best_model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(0.01))

history = best_model.fit(x_train,y_train, epochs = 100,validation_data=(x_validate,y_validate),verbose = 0)
a = np.min(history.history["loss"])
b = np.min(history.history["val_loss"])
c = best_model.evaluate(x_test,y_test, verbose = 2)

print()
print('best model has depth of', best_dep)
print('best model has width of', best_wid)
print("train loss (mean squared error, MSE) of best Model is :",a)
print("validation loss (mean squared error, MSE) of best Model is :",b)
print("test loss (mean squared error, MSE) of best Model is :",c)

8/8 - 0s - loss: 145.8887 - 27ms/epoch - 3ms/step

best model has depth of 5
best model has width of 15
train loss (mean squared error, MSE) of best Model is : 116.04651641845703
validation loss (mean squared error, MSE) of best Model is : 112.22354125976562
test loss (mean squared error, MSE) of best Model is : 145.8887481689453
